<a href="https://colab.research.google.com/github/szheng3/recommendation-system/blob/main/Explore_CQL/DLR2/SA2C_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self-Attentive Sequential Recommender (SASRec) + Supervised Advantage Actor-Critic (SA2C) Recommender on Retail Rocket Data with CQL loss
In this notebook, we train an SASRec-SA2C model to recommend a list of items to users in the dataset. We are using Retail Rocket data to train this model.

1. Clone the git repository containing all the source code

In [1]:

!git clone https://github.com/szheng3/recommendation-system.git


Cloning into 'recommendation-system'...
remote: Enumerating objects: 1540, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 1540 (delta 177), reused 291 (delta 168), pack-reused 1219
Receiving objects: 100% (1540/1540), 11.66 MiB | 8.90 MiB/s, done.
Resolving deltas: 100% (807/807), done.


2. Install required libraries

In [ ]:
#!pip install -r '/content/recommendation-system/Explore_CQL/requirements.txt'

In [3]:
!pip install trfl d3rlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached trfl-1.2.0-py3-none-any.whl (104 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.6/317.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 10.2 MB/s eta 0:00:00
  Created wheel for d3rlpy: filename=d3rlpy-1.1.1-cp310-cp310-linux_x86_64.whl size=1257801 sha256=aabacabd29073c5fec11938d4516867d90971cd6f3317707dba5c28014f5fc20
  Stored in directory: /root/.cache/pip/wheels/fb/44/c4/aab790777bd0d1358b9e8cc391621a4e0e2b10e7cd03406ad6
Successfully built d3rlpy


3. Download Retail Rocket events log from AWS S3 bucket to local

In [4]:
!wget https://aipi590.s3.amazonaws.com/events.csv -P '/content/recommendation-system/Explore_CQL/Data/RR_data'

--2023-05-03 09:52:00--  https://aipi590.s3.amazonaws.com/events.csv
Resolving aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)... 3.5.7.110, 52.216.40.177, 52.217.227.225, ...
Connecting to aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)|3.5.7.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94237913 (90M) [text/csv]
Saving to: ‘/content/recommendation-system/Explore_CQL/Data/RR_data/events.csv’

events.csv          100%[===================>]  89.87M  49.1MB/s    in 1.8s    

2023-05-03 09:52:02 (49.1 MB/s) - ‘/content/recommendation-system/Explore_CQL/Data/RR_data/events.csv’ saved [94237913/94237913]



4. Run script to pre-process data and generate replay buffer

In [5]:
!python '/content/recommendation-system/Explore_CQL/DLR2/src/gen_replay_buffer.py' --data='/content/recommendation-system/Explore_CQL/Data/RR_data'


2023-05-03 09:52:11.120535: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-03 09:52:11.176938: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 09:52:12.112583: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Starting to pre-process data...

Sorting and pickling data...

Splitting data into train, validation, and test sets...

Pickling train, validation, and test sets...

Calculating item popularity and storing as dictionary...

Generating replay buffer 

5. Run script to begin training and evaluate model. The model below is SASRec-SA2C. This run is without CQL. 

In [8]:
!python "/content/recommendation-system/Explore_CQL/DLR2/src/SA2C_v3_5.py" --model=SASRec --epoch=10 --data="/content/recommendation-system/Explore_CQL/Data/RR_data"

2023-05-03 10:01:14.500302: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-03 10:01:14.553825: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 10:01:15.577763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Not using CQL loss
2023-05-03 10:01:17.536452: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed 

5b. Run script to beging training and evaluate model. The model below is SASRec-SA2C. This is run with CQL and alpha factors of 1.0 and 0.5.

In [9]:

!python "/content/recommendation-system/Explore_CQL/DLR2/src/SA2C_v3_5.py" --model=SASRec --epoch=10 --CQL_alpha=1.0 --data="/content/recommendation-system/Explore_CQL/Data/RR_data"

2023-05-03 11:46:30.671475: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-03 11:46:30.724254: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 11:46:31.745776: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using CQL loss.
2023-05-03 11:46:33.629547: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a v

In [10]:

!python "/content/recommendation-system/Explore_CQL/DLR2/src/SA2C_v3_5.py" --model=SASRec --epoch=10 --CQL_alpha=0.5 --data="/content/recommendation-system/Explore_CQL/Data/RR_data"

2023-05-03 13:31:15.838967: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-03 13:31:15.891639: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 13:31:16.941371: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using CQL loss.
2023-05-03 13:31:18.893645: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a v